In [2]:
#importing the libraries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [4]:
#prepare the data
data = """Text generation is the task of generating meaningful text given a sequence of text as input.
It is widely used in applications such as chatbots, language translation, and text summarization.
Generating text requires a model to learn the patterns and structure of the input text to produce coherent and contextually relevant output."""


In [6]:
#Tokenize the text
tokenizer = Tokenizer() # create a vocabulary
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1  # +1 for the reserved 0 index

# Convert text to sequences of integers
input_sequences = []
for line in data.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [8]:
# Pad sequences to ensure they are of the same length
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Create predictors and labels
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = to_categorical(label, num_classes=total_words)


In [10]:
#build the LSTM model
model = Sequential()
model.add(Embedding(total_words, 10))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))


In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [14]:
model.fit(predictors, label, epochs=100, verbose=1)


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 3.5835
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.5798
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.5762
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.5718
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.5666
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.5607
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.5486
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.5315 
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.4981
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.4585
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.4334
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.4376
Epoch 13/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.4251
Epoch 14/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.3992
Epoch 15/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.3450
Epoch 16/100
2/2 ━

In [16]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=-1) # find the index of the word with the highest probality
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

seed_text = "AI"
generated_text = generate_text(seed_text, 20, max_sequence_len)
print(generated_text)


AI is is is is is the the the text given and of of text input input to produce coherent and
